## PS3

In [16]:
import numpy as np
from numpy.polynomial import Polynomial as P
import plotly
import plotly.plotly as py
import plotly.figure_factory as ff



def f(x):
    return (x-5)*np.exp(-(x/2-3))+10
    
def findw(f,lower,upper):
    slope=(f(upper)-f(lower))/upper-lower
    A=(slope/2)*(upper**2)-(slope*upper-f(upper))*upper-((slope/2)*(lower**2)-(slope*upper-f(upper))*lower)
    A/=(upper-lower)
    return [slope/A, (-slope*upper+f(upper))/A] 

def integrate(f,w,lower,upper,N):
    
    inputs=(upper-lower)*np.random.rand(N)+lower
    C=lower-((w[0]/2)*lower**2+w[1]*lower)
    p= [P([-i+C,w[1],w[0]/2]) for i in inputs]
    inverse_inputs=[]
    for i in p:
        if i.roots()[0]>=lower and i.roots()[0]<=upper:
            inverse_inputs.append(i.roots()[0])
        else: 
            inverse_inputs.append(i.roots()[1])
    inverse_inputs=np.array(inverse_inputs)
    outputsF=f(inverse_inputs)
    outputsW=w[0]*(inverse_inputs)+w[1]
    outputs=outputsF/outputsW
    FSUM2=(outputs**2).sum()
    var=(1/N)*(FSUM2/N-(outputs.sum()/N)**2)
    return (upper-lower)*outputs.sum()/N , var
    
l=np.arange(2,10.01,0.8)   
#print(l)
I_real=-16.673
N=[10,100,1000,10000]
results=[]
sigmas=[]
for k in N:
    I=0
    for i in range (0,len(l)-1):
        w=findw(f,l[i],l[i+1])
        temp,temp2=integrate(f,w,l[i],l[i+1],k)
        I+=temp
        sigma+=temp2
    I=I-80
    results.append(I)
    sigmas.append(sigma)
    
data_matrix = [["N", "I MC", "I Real", "I MC-I Real","sigma"],
               ["100", results[0], I_real,results[0]-I_real,sigmas[0]],
               ["1000", results[1], I_real,results[1]-I_real,sigmas[1]],
               ["10000", results[2], I_real,results[2]-I_real,sigmas[2]],
               ["100000", results[3], I_real,results[3]-I_real,sigmas[3]]]
plotly.tools.set_credentials_file(username='guneykan', api_key='Yu3MsgD6Zlfbb0B3S5Mx')
table = ff.create_table(data_matrix)
py.iplot(table)